1)Написать приложение, которое собирает основные новости с сайтов mail.ru и lenta.ru.

2) (Дополнительно) Собрать все новости со страницы https://yandex.ru/news/

Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

In [1]:
import requests
from lxml import html
from pprint import pprint
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client.news_database
collection = db.news

In [3]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/78.0.3904.97 Safari/537.36'}

In [4]:
url_mail = 'https://mail.ru/'

In [5]:
request = requests.get(url_mail, headers=header)

In [6]:
root = html.fromstring(request.text)
result_list = root.xpath('//div[contains(@class, "news-item")]/a[contains(@href, "https")]')

In [7]:
if result_list:
    for i in result_list:
        w = {}
        w['href'] = i.xpath('string(./@href)')
        w['text'] = i.text
        w['date'] = ' '
        w['source'] = 'mail.ru'
        collection.insert_one(w)

In [8]:
def search_news(date, collection):
    for news in collection.find({"date": {"$gt": ''}}):
        print(
            "-" * 79 + f"\n{news['text']}\n{news['href']}\n{news['date']}\n{news['source']}")

In [9]:
url_lenta = 'https://lenta.ru/'

In [10]:
request = requests.get(url_lenta, headers=header)

In [11]:
root = html.fromstring(request.text)
result_list = root.xpath('//section[@class="row b-top7-for-main js-top-seven"]/*/div[@class="item"]/a')

In [12]:
if result_list:
    for i in result_list:
        w = {}
        w['href'] = url_lenta + i.xpath('string(./@href)')
        w['text'] = i.xpath('string(./text())')
        w['date'] = i.xpath('string(./time/@datetime)')
        w['source'] = 'lenta.ru'
        collection.insert_one(w)

In [13]:
url_yandex = 'https://yandex.ru/news'

In [14]:
request = requests.get(url_yandex, headers=header)

In [15]:
root = html.fromstring(request.text)

In [16]:
result_list = root.xpath('//div[contains(@class, "story story_view")]')

In [17]:
if result_list:
    for i in result_list:
        w = {}
        iw = i.xpath('.//a[@class="link link_theme_black i-bem"]')
        w['href'] = url_yandex[:-5] + iw[0].xpath('string(./@href)')
        w['text'] = iw[0].xpath('string(./text())')
        w['date'] = i.xpath('.//div[@class="story__date"]/text()')
        w['source'] = 'yandex.ru'
        collection.insert_one(w)

In [18]:
search_news('', collection)

-------------------------------------------------------------------------------
None
https://news.mail.ru/incident/40169034/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
В РПЦ рассказали, какие пожертвования на храм принимать нельзя
https://news.mail.ru/society/40170019/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
Умер режиссер «Приключений Электроника» и «Чародеев»
https://news.mail.ru/society/40167664/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
Скандалы и отставки: социологи рассказали об ожиданиях россиян
https://news.mail.ru/society/40164981/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
Женщины стали чаще менять отчество на матроним
https://news.mail.ru/society/40167005/?frommail=10
 
mail.ru
-------------------------------------------------------------------